# Ismétlés előző óráról

## Típusok: `list`, `dict`,  `set`
 
 * `list` elemek sorozata (tömb). Indexelés pozícióval `[]`-ben.

* `dict` kulcs-érték párok. Indexelés kulccsal `[]`-ben.

* `set` halmaz. Nem lehet indexelni az elemeket, de végig lehet iterálni egy halmaz elemein `for` ciklussal.


In [ ]:
text = "ez egy hosszabb szöveg"
for char in set(text):
    print(repr(char), end=" ")
print()

## Függvények

- Függvényhez lehet dokumentációs sztringet adni!
- Az argumentumokat és a visszatérési értéket típus annotációval lehet ellátni.
- Lehet változó számú paraméterű függvényt írni. 
  Ha a paraméter listában  `*args`, `**kwargs` szerepel, 
  akkor az

  * extra pozicionális paraméterek az `args` paraméterben lesznek `tuple`-ként.

  * extra névvel átadott paraméterek a `kwargs` szótárban lesznek.

## Függvények

- Függvény hívás során is használható `*` és `**` operátor. 
  
```python
arglist = [1, 2, 3]
f(*arglist) # azzal ekvivalens, hogy f(1, 2, 3)
  
argdict = {"a": 1, "b": 2, "c": 3}
f(**argdict) # azzal ekvivalens, hogy f(a=1, b=2, c=3)
  
f(-1, 0, *arglist, **argdict) # azzal ekvivalens, hogy f(-1, 0, 1, 2, 3, a=1, b=2, c=3)
```

- Értékadás baloldalán  is szerepelhet a `*` konstrukció.

# Mi történik `for` ciklus alkalmazásakor

In [ ]:
seq = range(5)
for item in seq:
    print(item, end=" ")

- Először a `seq` sorozatra az `iter` függvényt alkalmazzuk. Ezzel egy listából, szótárból,  halmazból egy iterátor keletkezik.

- Ezután az kapott iterátorra a `next` függvényt alkalmazzuk. Ez minden függvényhívásra a sorozat következő elemével válaszol.

- A kapott értéket az `item` változónak adjuk értékül és a ciklus törzsét végrehajtjuk. 

A példában látott:

```python
seq = range(5)
for item in seq:
    print(item, end=" ")
```

kódrészlet nagyjából a következővel ekvivalens:

In [ ]:
seq = range(5)
it = iter(seq)
while (item := next(it, None)) is not None:
    print(item, end=" ")

## Generátor függvények

- Mit kapunk, ha a `for` ciklusban használható függvényeket közvetlenül meghívjuk?

- Hogyan tudunk a sorozat elemeihez hozzáférni?

- Tudunk-e ehhez hasonló függvényt írni, ami sorozatot állít elő és `for` ciklusban használható?

In [ ]:
print(range(10))
print(enumerate(range(10)))
print(zip(range(5), "alma"))
print(reversed(range(5)))

## `iter` és `next`


In [ ]:
seq = range(10)
it = iter(seq)
print(f"{seq=}\n{it=}\n{next(it)=}, {next(it)=}, {next(it)=}")

In [ ]:
it = iter(enumerate(seq))
print(f"{it=}\n{next(it)=}, {next(it)=}, {next(it)=}")

Hasonlóan működik `zip` és `reversed` esetén is. Ezek a függvények iterátort adnak vissza. `iter` alkalmazása felesleges, de nem vezet hibára.

## `yield` kulcsszót tartalmazó függvények

In [ ]:
def f():
    yield 1
    yield -1
    yield 23
    
print(f"{f=},\n{f()=},\n{iter(f())=},\n{next(f())=}") 

In [ ]:
for i, x in enumerate(f(), 1):
    print(f"{i}. iteráció értéke: {x}")

## Mi történik generátor függvény végrehajtásakor?

In [ ]:
it = f()
print(f"{it=}, {next(it)=}, {next(it)=}, {next(it)=}, {next(it, None)=}")

- Amikor a függvényt meghívjuk létrejön egy iterátor. <!-- , ez az it változó értéke.  -->A függvény törzséből semmi nem kerül végrehajtásra.

- Amikor a `next` függvényt először alkalmazzuk a függvény törzse az első `yield` statement-ig végrehajtódik. A `next` függvény visszatérési értéke a `yield` statement utáni kifejezés értéke.

- A következő `next` meghívásakor az `f` függvény végrehajtása folytatódik, onnan ahol abbamaradt, a következő `yield` statement-ig. Ebből lesz a visszatérési érték. És így tovább.

## Leállás, `StopIteration`

- Ha a `next` hívás hatására a generátor függvény nem ütközik újabb `yield` statement-be, akkor a sorozat kimerült és ezt 
`StopIteration` hibával jelzi a Python.

- A `next` függvénynek megadhatunk az iterátor mellett egy második argumentumot is. Ekkor `StopIteration` hiba helyett ezt fogjuk látni értékként, ha elfogyott  sorozatunk.


In [ ]:
def f():
    yield 1
    yield 2
    
it = f()
print(f"{next(it, None)=}, {next(it, None)=}, {next(it, -1)=}")

## Mi történik itt?

In [ ]:
def f():
    yield 1
    yield -1
    yield 23
    
it1 = f()
it2 = f()
next(it1)
print(f"{[*zip(it1, it2)]=}")

Egy generátor függvény meghívásakor mindig új iterátor keletkezik. Ezek ugyanazt a sorozatot állítják elő, de egymástól függetlenül haladnak végig azon.

## Végtelen iterátor

In [ ]:
def fibonacci_it():
    a, b = 0, 1
    yield a
    while True:
        yield b
        a, b = b, a+b
        
print([*zip(range(10), fibonacci_it())])

## Zárójeles `for` ciklusok = generátor kifejezés 

In [ ]:
seq = (x**2 for x in range(5))
print(f"{type(seq)=}, {seq=}")

In [ ]:
print(f"{max(seq, default=-1)=}, {max(seq, default=-1)=}")

In [ ]:
seq = (x**2 for x in range(5))
print(f"{any(seq)=}, {min(seq, default=-1)=}")

# Double ended queue (lista segítségével)

- A célunk az, hogy implementáljunk egy `list`-hez hasonló típust.

- Láttuk, hogy `list` esetében az `.append` és `.pop` műveletek átlagban konstans időt vesznek igénybe, holott időnként a teljes listát másolni kell.

  Ez úgy érhető el, hogy a szükségesnél több helyet foglalunk le és csak akkor másolunk, ha elfogyott a hely.

- Olyan típust szeretnénk, ami a listához hasonló, de mindkét végéről lehet gyorsan törölni, ill. hozzáfűzni. 

### Egy tipikus feladat `deque` alkalmazására 

> Adott egy számokból álló lista `numbers` és egy `k` pozitív egész, `1 <= k <= len(numbers)`. Számítsuk ki a `k` hosszú blokkok maximumait (`n - k + 1` hosszú lista).

### Ötlet.

- Az értékeket egy listában tároljuk, de a lista hosszabb mint a tárolt értékek száma (legalábbis kezdetben).

- Külön változóban tároljuk, hogy hol kezdődnek a tárolt értékek, és hány értéket tárolunk. 

- Ha van hely akkor a hozzáfűzés egyszerűen a kezdeti pozíció mozgatásával, ill. a hossz változtatásával megoldható. A törlés hasonlóan.

- Ha csak ,,ritkán'' kell másolni, akkor a művelet igény átlagban konstans. 

In [21]:

def init_queue(size=None):
    if size is None:
        size = 8
    size = max(size, 8)
    return [0]*2 + [None]*size

def str_queue(lst):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    elements = [lst[2 + ((i+start) % capacity)] for i in range(size)] 
    return f"deque of size {size}: {elements}"   


In [ ]:
q = init_queue()
print(f"{str_queue(q)=}, {q=}")

In [23]:
def increase_size(lst):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    lst.extend(None for _ in range(max(capacity//8, 1)))
    new_capacity = len(lst)-2
    for i in range(size):
        lst[2+((start+i) % new_capacity)] = lst[2+((start+i) % capacity)]
        
def append(lst, item):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    if size == capacity:
        increase_size(lst)
        capacity = len(lst) - 2
    lst[2+((start+size) % capacity)] = item
    size += 1
    lst[1] = size

In [24]:
def append_left(lst, item):
    start, size, capacity = lst[0], lst[1], len(lst) - 2
    if size == capacity:
        increase_size(lst)
        capacity = len(lst) - 2
    start = (start-1) % capacity 
    size += 1
    lst[0], lst[1], lst[2+start] = start, size, item 

## Kis próba

In [ ]:
q = init_queue(5)
for i in range(5):
    if i % 2:
        append_left(q, i)
    else:
        append(q, i)
print(f"{str_queue(q)=}, {q=}")


In [26]:
def decrease_size(lst):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    new_capacity = max(8, capacity-capacity//8)
    if new_capacity == capacity:
        return
    values = [lst[2+((start+i) % capacity)] for i in range(size)]
    lst.clear() 
    lst.extend((0, len(values)))
    lst.extend(values)
    lst.extend(None for _ in range(new_capacity-len(values)))

def pop(lst):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    if size <= 0:
        print("Error: cannot pop from empty queue!")
    size -= 1
    value = lst[2 + ((start + size) % capacity)]
    lst[1] = size
    if capacity - size >= capacity//4:
        decrease_size(lst)
    return value
    

def pop_left(lst):
    start, size, capacity = lst[0], lst[1], len(lst)-2
    if start <= 0:
        print("Error: cannot pop from empty queue!")
    value = lst[2 + (start % capacity)]
 
    start, size = start+1, size-1
    lst[0], lst[1] = start, size
    if capacity - size >= capacity//4:
        decrease_size(lst)
    return value

# Osztály, objektum



Objektum: adat és a hozzátartozó műveletek együtt. 

Pythonban az 

- osztály a típust jelenti ez definiálja a műveleteket, 

- az objektum az osztály egy példánya. Az objektum tartalmazza az adatot.

Az előző példában az implementáció nem érdekes. 

Négy műveletünk volt `append`, `append_left`, `pop`, `pop_left`.

Az objektumot egy listával reprezentáltuk.


## Egyszerű példák 

In [ ]:
# Person egy osztály
class Person:
    pass

# `p` objektum a Person osztály egy példánya
p = Person()
# print(p)
# a p példányhoz dinamikusan lehet attribútumokat adni
p.name = "Anna"
p.age = 25

print(f"{p=}")
print(f"{p.name=}")
print(f"{p.age=}")
print(f"{p.__dict__}")

## Inicializálás

Minden osztálynak lehet `__init__` metódusa. Ez inicializálja az osztály egy példányát. 
Így az attribútumok nem utólag lesznek hozzáadva az adott példányhoz. 

In [ ]:
class Person:
    # Minden metódus első paramétere maga az osztálypéldány
    def __init__(this, name, age):
        this.name = name
        this.age = age
    
    
p = Person(name="Ann", age=25) 
print(f"{p=}")
print(f"{p.name=}")
print(f"{p.age=}")

## Műveletek, metódusok

In [32]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    

In [ ]:
p = Person("Ann", 25)

p.introduction()

## Szöveges reprezentáció

In [38]:
# print(p)

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"Person({self.name!r}, {self.age})"
#     This is {self.name}, {self.age} year(s) old."
      
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    
p = Person("Ann", 25)
print(p)

Van egy `__str__` metódus is a `__repr__` mellett. Az utóbbi általában magunknak szól, az előbbi a végfelhasználónak. Ha osztályt írunk, `__repr__` metódust mindig érdemes implementálni.

OOP-ből ismert fogalmak lehetnek a `public`, `protected` és `private` attribútumok fogalma. Pythonban nincs kulcsszó ezekre, egyszerűen egy nevezéktani konvenció biztosítja, hogy melyik attribútum érhető el kívülről, és melyeket szeretnénk protected-nek vagy privátnak tekinteni.

Más nyelvekből ismerős `getter` és `setter` függvényeket lehet ugyan írni, de ne felejtsük, hogy ez itt Python, nincs kikényszerítve, hogy ilyenek írjunk és nem is feltétlenül szokás.

In [ ]:
class Person:
    class_variable = 123
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def __repr__(self):
        return f"This is {self.name}, {self.age} year(s) old."
    def is_pensioner(self):
        return self.age >= 65
    
p = Person(name="Ann", age=25)  
print(f"{p.is_pensioner()=}")
print(f"{p.class_variable=}")
print(f"{Person.class_variable=}")

## Bele tudunk-e nézni egy osztályba, objektumba?

In [ ]:
person = Person(name="Anna", age=19)

print(f"{vars(person)=}")

vars(Person)

Amikor az egyed egy `attribútum`-át kérdezzük le, vagy állítjuk be, akkor elindul egy keresés. Az egyed és az osztály változói (`vars` által visszaadott érték) között. Ha megtaláltuk az attribútumot azzal dolgozunk tovább.



## Itt mi történik?

In [ ]:

person = Person(name="Anna", age=19)
print(f"{person.class_variable=}")
# person.class_variable = 150
# print(f"{vars(person)}\n{person.class_variable=}")
Person.class_variable = 120
print(f"{person.class_variable=}, {Person.class_variable=}")

In [ ]:
print(f"{vars(person)=}")
vars(Person)

## És itt mi történik?

In [ ]:
person = Person(name="Anna", age=19)
person.is_pensioner = True #lambda: True
print(f"{person.is_pensioner()=}")
print(f"{vars(person)=}")

In [ ]:
person = Person(name="Anna", age=19)
Person.is_pensioner = lambda self: self.age > 18
person.is_pensioner()

Egy osztályhoz utólag is lehet műveleteket fűzni. 

Csak azért mert lehet, nem biztos, hogy kell is.

# További példa: ,,bankszámla'' osztály

In [55]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
    def __repr__(self):
        return f"BankAccount(account_id={self._id}, initial_amount={self._amount})."
    def amount(self):
        return self._amount
    def deposit(self, money):
        self._amount += money
        return self._amount
    def withdraw(self, money):
        if self._amount < money:
            print(f"Not enough money on your account: requested {money} but your balance is {self.amount()}.")
            return 0
        self._amount -= money
        return money

In [ ]:
account = BankAccount(123)

account.deposit(100)
account.deposit(200)
_ = account.withdraw(250)
print(account)
print(account.amount())

Ez a számla nem túl biztonságos, mert az `_amount` mezőt lehet kívülről manipulálni, csak a konvención és az én jóindulatomon múlt, hogy titokban ne adjak pénzt közvetlenül ehhez a mezőhöz.

In [ ]:
account = BankAccount(123)
account._amount = 10000

print(account.amount())

Pythonban lehet ennél szigorúbb hozzáférést adni a tényleg privátnak gondolt mezőkhöz, illetve megfelelő settereket és gettereket itt is be lehet állítani 
<!-- , de mi most ennél tovább nem megyünk. -->

In [59]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
    def __repr__(self):
        return f"BankAccount(id={self._id}, amount={self._amount})."
    
    @property
    def amount(self):
        return self._amount
    
    @amount.setter
    def amount(self, value):
        print("use the deposit or withraw method to change your balance!")
        
    def deposit(self, money):
        self._amount += money
        return self._amount
        
    def withdraw(self, money):
        if self._amount < money:
            print(f"Not enough money on your account: requested {money}, but your balance is {self.amount}.")
            return 0
        self._amount -= money
        return money

In [ ]:
account = BankAccount(123)
account.deposit(100)

print(account)
print(account.amount)

In [ ]:
account.amount = 1000

 **Házi feladat.** Egészítsük ki  `Temperature` osztályt! 

In [63]:
class Temperature: ## csak ezért van így, hogy kiférjen a diára!!!
    def __init__(self, celsius=0): 
        self.celsius = celsius
    def __repr__(self): 
        return f"{self.celsius} ℃ [{self.fahrenheit} ℉]"
    @property
    def celsius(self): 
        pass
    @celsius.setter
    def celsius(self, value): 
        pass
    @property
    def fahrenheit(self): 
        pass
    @fahrenheit.setter
    def fahrenheit(self, value): 
        pass

In [ ]:
t = Temperature()
t.celsius = 25
print(f"{t.fahrenheit=}, {t=}")

t.fahrenheit = 101
print(f"{t.celsius=}, {t=}")


Itt azt várnánk, hogy a `t.fahrenheit` értéke $32+25*9/5=77$ és `t` szöveges reprezentációja `25 ℃ [77.0 ℉]`

## Miért használunk osztályokat? 

Osztályokat nem kötelező használni, de előbb-utóbb találkozunk velük, mások kódjaiban, vagy könyvtárakból való importok esetén. Néhány érv a használatuk mellett:

* Valamilyen módon összetartozó adatokat akarunk eltárolni egységbe zárva (encapsulation)

* Egy állapotot kell nyilvántartani, illetve ezt az állapotot kell tudnunk megfelelő módon megváltoztatni.

* Az adatot csak meghatározott módon lehessen manipulálni, a felhasználónak nem kell tudnia, hogy mi a belső implementáció (absztrakció)

* Hierarchiába szervezhető adattípusok vannak (öröklődés)
<!-- 
* Gépi tanulásban gyakori, hogy egy algoritmusnak több (hyper) (algoritmustól függő) paramétere van, ugyanakkor a modell illesztés és predikció ugyanolyan módon történik.

```python
from sklearn.linear_model import Ridge

model = Ridge(alpha=0.1, fit_intercept=False, solver='saga', max_iter=5)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
``` -->

# Double Ended Queue osztályként

In [ ]:
class DeQueue:
    min_size = 8
    upfactor = 1.125
    
    def __init__(self, size=None):
        if size is None:
            size = self.min_size
        size = max(size, self.min_size)
        
        self._values = [0]*size
        self._start = 0 
        self._size = 0


Ebben a részletben `min_size` és `upfactor` a típus minden egyedére közös változó. Nem az egyedben, hanem a típusban van tárolva.

In [ ]:
def dq_str(self):
    return f"{type(self).__name__} with size {self._size}."

def dq_repr(self):
    start, size = self._start, self._size
    values, n = self._values, len(self._values)
    elements = [values[(i+start) % n] for i in range(size)]
    return f"{type(self).__name__}({elements})"

## csak azért csináljuk így, hogy a diára ráférjen!!!
DeQueue.__str__ = dq_str
DeQueue.__repr__ = dq_repr

In [ ]:
q = DeQueue()
print(f"{q=!r}\n{q=!s}")

In [ ]:
def dq_reset(self, new_size):
    new_values = [0]*new_size
    values, n = self._values, len(self._values)
    start = self._start 
    for i in range(self._size):
        new_values[i] = values[(start+i) % n]
    self._values = new_values
    self._start = 0

DeQueue._reset = dq_reset

A `_reset` művelet neve aláhúzással kezdődik. Konvenció szerint ezt kívülről nem illik használni. 
Nincs ellenőrzés a `new_size`-ra. 

In [ ]:
def dq_check_size(self):
    if self._size >= len(self._values):
        new_size = max(len(self._values)+1, int(len(self._values)*self.upfactor))
        self._reset(new_size)
    elif self.upfactor*self.upfactor*self._size <= len(self._values):
        new_size = max(self.min_size, int(len(self._values)/self.upfactor))
        if new_size != self._size:
            self._reset(new_size)
    
DeQueue._check_size = dq_check_size

In [ ]:
q = DeQueue()
q._size = len(q._values)
q._check_size()
print(f"növelés után: {len(q._values)=}", end=", ")
q._size = 5
q._check_size()
print(f"csökkentés után: {len(q._values)=}")

In [ ]:
def dq_append(self, item):
    self._check_size()
    self._values[(self._start+self._size) % len(self._values)] = item
    self._size += 1


def dq_pop(self):
    self._size -= 1
    value = self._values[(self._start+self._size) % len(self._values)] 
    self._check_size()
    return value

DeQueue.append = dq_append
DeQueue.pop = dq_pop

In [ ]:
q = DeQueue()
q.append(1)
q.append(2)
print(f"{q=}, {q.pop()=}, {q=}")

In [ ]:
def dq_append_left(self, item):
    self._check_size()
    self._start = (self._start - 1) % len(self._values)
    self._values[self._start] = item
    self._size += 1
    
def dq_pop_left(self):
    value = self._values[self._start % len(self._values)]
    self._start = (self._start + 1) % len(self._values)
    self._size -= 1
    self._check_size()
    return value

DeQueue.append_left = dq_append_left
DeQueue.pop_left = dq_pop_left

In [ ]:
q.append_left(3)
print(f"{q=}, {q.pop_left()=}, {q=}")

## Oldjuk meg a motiváló feladatot.

> Adott egy számokból álló lista `numbers` és egy `k` pozitív egész, ahol `1 <= k <= len(numbers)`. Számítsuk ki a `k` hosszú blokkok maximumait. Az eredmény `n - k + 1` hosszú lista.

Van egy `double ended queue`-nk. Egy hatékonyabb implementáció megtalálható a `collections` könyvtárban `deque` névvel, azzal fogunk dolgozni.

### Először brute force megoldás

- Ezzel ellenőrizhetjük a ,,hatékony'' megoldásunkat.
- Össze tudjuk hasonlítani a futási időket.

In [65]:
def window_maxes_slow(numbers, k):
    return [max(numbers[j] for j in range(i, i+k)) for i in range(len(numbers)-k+1)]       

In [ ]:
window_maxes_slow([1, 2, 3, 2, 1], 2)

In [70]:
import random
random_numbers = [random.random() for _ in range(15_000)]

In [ ]:
%time maxes = window_maxes_slow(random_numbers, 7500)

### Mit kell megjegyezni az utolsó $k$ számból?

In [ ]:
import matplotlib.pyplot as plt
colors = ["blue", "red"]
is_max = [colors[x==max(random_numbers[i:20])] for i, x in enumerate(random_numbers[:20])]
plt.scatter(x=range(20), y=random_numbers[:20], c=is_max) 
plt.grid()
plt.show()

### Mit kell megjegyezni az utolsó $k$ számból?

- Legyen `numbers[i-k:i]` `i>=k` az `i`. ablak. A feladat a maximumok összegyűjtése.

- Minden ablakra jegyezzünk fel egy sorozatot a lehetséges maximum értékekből.
  Ha `w` az ablak, akkor 
  
  * a sorozat első eleme legyen az első maximum érték.
  * a következő elem a tőle jobbra lévők közül az első maximum érték, stb. 
  
  Az adott ablak maximális értéke, a hozzá tartozó sorozat első eleme.
  
- Ha az ablakot eggyel jobbra mozgatjuk, akkor 
  * a korábbi maximum érték kikerülhet az ablakból
  * az új érték a sorozat néhány eleménél nagyobb lehet. Ezeket törölni kell. 
  * végül az új érték a tőle jobbra állóktól nem kisebb, így hozzá kell fűzni a sorozathoz.
  


## Ugyanez kóddal

In [74]:
from collections import deque

def window_maxes(numbers, k):
    queue = deque()
    maxes = []
    for i, num in enumerate(numbers):
        if i >= k and queue[0] == numbers[i-k]:
            queue.popleft()
        while queue and queue[-1] < num:
            queue.pop()
        queue.append(num)
        if i >= k-1:
            maxes.append(queue[0])
    return maxes

In [75]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

def test_window_maxes():
    for _ in range(20):
        random_numbers = [random.randint(0, 1000)  for _ in range(200)]
        assert window_maxes_slow(random_numbers, 100) == window_maxes(random_numbers, 100)


## Futási idők

In [ ]:
print(f"{len(random_numbers)=}")
k = len(random_numbers)//2
%time maxes = window_maxes_slow(random_numbers, k)
%time maxes = window_maxes(random_numbers, k)

## `deque` helyett `list` típussal

In [78]:
def window_maxes_with_list(numbers, k):
    queue = []
    maxes = []
    for i, num in enumerate(numbers):
        if i >= k and queue[0] == numbers[i-k]:
            queue.pop(0)
        while queue and queue[-1] < num:
            queue.pop()
        queue.append(num)
        if i >= k-1:
            maxes.append(queue[0])
    return maxes


## Teszt

In [ ]:
%%ipytest

def test_window_maxes_list():
    for i in range(1, 20):
        n = i*1000
        numbers = [random.randint(0, 10000)  for _ in range(n)]
        assert (
            window_maxes_with_list(numbers, n//2) == 
            window_maxes(numbers, n//2)
        )

## Futási idő

In [ ]:
n = 50_000
random_numbers = [random.random() for _ in range(n)]
print(f"{len(random_numbers)=:_}")
%timeit -n 10 maxes = window_maxes_with_list(random_numbers, n//2)
%timeit -n 10 maxes = window_maxes(random_numbers, n//2)

Lehet, hogy a lista típus jobb mint gondoltuk?